In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('TransacoesClientes').getOrCreate()

transacoes = spark.read.csv('transacoes.csv', header=True, inferSchema=True)
clientes = spark.read.csv('clientes.csv', header=True, inferSchema=True)

transacoes.printSchema()
clientes.printSchema()

transacoes.show(5)
clientes.show(5)

root
 |-- id_transacao: integer (nullable = true)
 |-- id_cliente: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- data_transacao: date (nullable = true)
 |-- categoria: string (nullable = true)

root
 |-- id_cliente: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- cidade: string (nullable = true)

+------------+----------+-----+--------------+-----------+
|id_transacao|id_cliente|valor|data_transacao|  categoria|
+------------+----------+-----+--------------+-----------+
|           1|         1|100.0|    2023-11-01|    Compras|
|           2|         2|200.0|    2023-11-02|Alimentação|
|           3|         3|150.0|    2023-11-03|    Compras|
|           4|         4|300.0|    2023-11-01|     Viagem|
|           5|         5| 50.0|    2023-11-04|Alimentação|
+------------+----------+-----+--------------+-----------+

+----------+-----+-----+--------------+
|id_cliente| nome|idade|        cidade|
+----------

In [ ]:
df_merged = transacoes.join(clientes, on='id_cliente', how='inner')

df_merged.show(5)

+----------+------------+-----+--------------+-----------+-----+-----+--------------+
|id_cliente|id_transacao|valor|data_transacao|  categoria| nome|idade|        cidade|
+----------+------------+-----+--------------+-----------+-----+-----+--------------+
|         1|           1|100.0|    2023-11-01|    Compras| João|   35|     São Paulo|
|         2|           2|200.0|    2023-11-02|Alimentação|Maria|   25|Rio de Janeiro|
|         3|           3|150.0|    2023-11-03|    Compras| José|   40|      Curitiba|
|         4|           4|300.0|    2023-11-01|     Viagem|  Ana|   28|     São Paulo|
|         5|           5| 50.0|    2023-11-04|Alimentação|Paulo|   50|Belo Horizonte|
+----------+------------+-----+--------------+-----------+-----+-----+--------------+



In [ ]:
from pyspark.sql.functions import col, sum as _sum

df_total = df_merged.groupBy('id_cliente').agg(_sum('valor').alias('total_transacoes'))
df_total.show(5)

+----------+----------------+
|id_cliente|total_transacoes|
+----------+----------------+
|         1|           100.0|
|         3|           150.0|
|         5|            50.0|
|         4|           300.0|
|         2|           200.0|
+----------+----------------+



In [ ]:
cliente_top = df_total.orderBy(col('total_transacoes').desc()).limit(1)

cliente_top.show()